In [1]:
import gensim
import json
import tqdm
import operator

### load queries

In [2]:
# read the queries
queries = {}
with open('../data/robust/stopped_queries_lower.txt') as f:
    for line in tqdm.tqdm(f):
        q_id, q_text = line.strip().split('\t')
        q_id = int(q_id)
        queries[q_id] = q_text

250it [00:00, 26100.21it/s]


## Load Glove 

In [3]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_dir = '/mnt/scratch2/shnaseri/dataset/glove/glove.6B/'
glove2word2vec(glove_input_file=glove_dir+"glove.6B.300d.txt", word2vec_output_file=glove_dir + "gensim_glove_vectors_300d.txt")

(400000, 300)

In [4]:
glove_model = gensim.models.KeyedVectors.load_word2vec_format(glove_dir+"gensim_glove_vectors_300d.txt",binary=False)

### Similarity between the query q with the whole collection words

In [5]:
most_sim_terms_per_query = []
for q in tqdm.tqdm(queries):
    temp_dict = {}
    temp_dict['topicNumber'] = str(q)
    try:
        sim_terms = glove_model.most_similar(positive=queries[q].lower().split(' '), topn=1000)
        arr = []
        for t in sim_terms:
            arr.append({'word':t[0], 'weight':t[1]})
        temp_dict['terms'] = arr
        most_sim_terms_per_query.append(temp_dict)
    except:
        print(q, queries[q])

100%|██████████| 250/250 [00:13<00:00, 18.93it/s]


### Similarity between the query q and the terms in the topk PRF documents

In [6]:
def load_run(path):
    result = {}
    with open(path) as f:
        for line in tqdm.tqdm(f):
            q, _, d, rank, score, method = line.strip().split(' ')
            q = int(q)
            rank = int(rank)
            score = float(score)
            result.setdefault(q, {})
            result[q][d] = score
    return result

In [7]:
def get_topk_ret_doc_p_query(ret_result, topk):
    sorted_run = {}
    for q in ret_result:
        sorted_docs = sorted(ret_result[q].items(), key=operator.itemgetter(1), reverse=True)
        sorted_run[q] = sorted_docs[:topk]
    return sorted_run

In [8]:
def get_sim_terms_json_format(terms):
    terms_json_format = []
    for t in terms:
        terms_json_format.append({'word':t[0], 'weight':str(t[1])})
    return terms_json_format

In [9]:
# load the text of bm25 retrieved documents from a json file in the format {doc_id:doc_text}
bm25_ret_docs_file = '/path/to/bm25_document_text.json'
bm25_ret_docs_text = json.load(open(bm25_ret_docs_file))

In [10]:
# load the bm25 run on robust collection in the trec format
bm25_run_file = '/path/to/bm25_run.run'
bm25_run = load_run()

235963it [00:00, 428755.11it/s]


In [11]:
# Get the topk documents (PRF documents)
topk_bm25_run = get_topk_ret_doc_p_query(bm25_run, 10)

In [12]:
# extract the terms of the topk prf documents

topk_res_terms = {}
# iterates over all queries
for q in bm25_run:
    # iterates over the topk retrieved docs
    terms = set()
    for doc in topk_bm25_run[q]:
        doc_id = doc[0]
        # get the terms of retrieved doc
        doc_terms = set(bm25_ret_docs_text[doc_id].strip().split(' '))
        terms = terms.union(doc_terms)
        topk_res_terms[q] = terms

In [13]:
# find the similarity between query q and the topk terms of the PRF document
most_sim_terms_per_query = []
for q in queries:
    print(q, queries[q])
    temp_dict = {}
    temp_dict['topicNumber'] = str(q)
    try:
        arr = []
        c = 0 
        for t in topk_res_terms[q]:
            # Some terms in the PRF documents may not be in the glove vocabulary
            if t not in glove_model.vocab:
                print(t, ' not in glove vocab')
                continue
            c += 1
            sim_score = glove_model.n_similarity(queries[q].strip().lower().split(' '), [t])
            arr.append([t, sim_score])
        sorted_arr = sorted(arr, key=lambda x:x[1], reverse=True)
        terms_json_format = get_sim_terms_json_format(sorted_arr)
        temp_dict['terms'] = terms_json_format
        most_sim_terms_per_query.append(temp_dict)
        print('----------------------------------------------------------------')
    except Exception as e:
        print(e)
        print(q, queries[q])

301 international organized crime
bakatin  not in glove vocab
dekrety  not in glove vocab
counternarcotic  not in glove vocab
loeschnak  not in glove vocab
tanomento  not in glove vocab
preinvestigations  not in glove vocab
dm102  not in glove vocab
voel  not in glove vocab
dm250  not in glove vocab
nadein  not in glove vocab
werthebach  not in glove vocab
wolny  not in glove vocab
----------------------------------------------------------------
302 poliomyelitis post polio
anganwadis  not in glove vocab
nathani  not in glove vocab
diarrohoeal  not in glove vocab
unimmunized  not in glove vocab
polymyxin  not in glove vocab
shaktisinh  not in glove vocab
tunzale  not in glove vocab
athit  not in glove vocab
09494  not in glove vocab
kasumova  not in glove vocab
chhea  not in glove vocab
1yes  not in glove vocab
1no  not in glove vocab
debilities  not in glove vocab
bounkhouang  not in glove vocab
phatthaya  not in glove vocab
urairat  not in glove vocab
--------------------------------

s1ws  not in glove vocab
zanville  not in glove vocab
civilizers  not in glove vocab
terrordome  not in glove vocab
----------------------------------------------------------------
310 radio waves brain cancer
quackeries  not in glove vocab
sively  not in glove vocab
hoppszallern  not in glove vocab
hopitals  not in glove vocab
spectroscopists  not in glove vocab
myelography  not in glove vocab
nrpb  not in glove vocab
phosphocreatine  not in glove vocab
diasonics  not in glove vocab
diasonic  not in glove vocab
dimentia  not in glove vocab
megabrain  not in glove vocab
mopt  not in glove vocab
noninvasively  not in glove vocab
chemicalk  not in glove vocab
cartilege  not in glove vocab
stehbens  not in glove vocab
buyline  not in glove vocab
yuppiedom  not in glove vocab
noninva  not in glove vocab
jected  not in glove vocab
innerquest  not in glove vocab
experi  not in glove vocab
haimovich  not in glove vocab
----------------------------------------------------------------
311 indus

lf32  not in glove vocab
ekerdt  not in glove vocab
zapico  not in glove vocab
asturia  not in glove vocab
4bn  not in glove vocab
lossmaking  not in glove vocab
lf35  not in glove vocab
lf53  not in glove vocab
serps  not in glove vocab
macminn  not in glove vocab
br19  not in glove vocab
1nn  not in glove vocab
lf49  not in glove vocab
lf39  not in glove vocab
5bn  not in glove vocab
aleene  not in glove vocab
marmonte  not in glove vocab
dilnot  not in glove vocab
----------------------------------------------------------------
319 new fuel sources
846b  not in glove vocab
08625  not in glove vocab
80mw  not in glove vocab
846c  not in glove vocab
2911  not in glove vocab
nonutility  not in glove vocab
250mw  not in glove vocab
1project  not in glove vocab
naaqs  not in glove vocab
exceedance  not in glove vocab
200kw  not in glove vocab
1agency  not in glove vocab
1final  not in glove vocab
3312  not in glove vocab
1date  not in glove vocab
300mw  not in glove vocab
11mw  not in gl

6bn  not in glove vocab
610m  not in glove vocab
280m  not in glove vocab
273m  not in glove vocab
75bn  not in glove vocab
oustees  not in glove vocab
5bn  not in glove vocab
wagstyl  not in glove vocab
beanos  not in glove vocab
----------------------------------------------------------------
332 income tax evasion
dm4bn  not in glove vocab
790m  not in glove vocab
y5m  not in glove vocab
7bn  not in glove vocab
y7bn  not in glove vocab
76bn  not in glove vocab
y26m  not in glove vocab
dm73bn  not in glove vocab
dm17bn  not in glove vocab
dm7bn  not in glove vocab
dm11bn  not in glove vocab
dm24bn  not in glove vocab
74bn  not in glove vocab
y118m  not in glove vocab
dr300bn  not in glove vocab
dm9  not in glove vocab
dr1m  not in glove vocab
dr400bn  not in glove vocab
dm3bn  not in glove vocab
contral  not in glove vocab
144m  not in glove vocab
dr50bn  not in glove vocab
dm75bn  not in glove vocab
y50m  not in glove vocab
-----------------------------------------------------------

phunsom  not in glove vocab
bevering  not in glove vocab
7122  not in glove vocab
khing  not in glove vocab
flpma  not in glove vocab
drk  not in glove vocab
4191  not in glove vocab
8090  not in glove vocab
14626  not in glove vocab
85027  not in glove vocab
layu  not in glove vocab
----------------------------------------------------------------
341 airport security
babatope  not in glove vocab
zaidman  not in glove vocab
aremu  not in glove vocab
18384  not in glove vocab
bline  not in glove vocab
dhunkaal  not in glove vocab
anabodhe  not in glove vocab
sc183  not in glove vocab
shurm  not in glove vocab
guardforce  not in glove vocab
4910  not in glove vocab
defranchised  not in glove vocab
qalaad  not in glove vocab
----------------------------------------------------------------
342 diplomatic expulsion
ostgaard  not in glove vocab
mashat  not in glove vocab
movahed  not in glove vocab
dagsrevyen  not in glove vocab
pakhtusov  not in glove vocab
unware  not in glove vocab
rapite

...

769851  not in glove vocab
longimembris  not in glove vocab
macularius  not in glove vocab
bl4  not in glove vocab
4310  not in glove vocab
4401  not in glove vocab
uncuddly  not in glove vocab
unknotting  not in glove vocab
744707  not in glove vocab
pohakuloa  not in glove vocab
786728  not in glove vocab
macrodactylum  not in glove vocab
20737  not in glove vocab
bl2  not in glove vocab
coolpassion  not in glove vocab
bontebok  not in glove vocab
20782  not in glove vocab
4916  not in glove vocab
787484  not in glove vocab
reimports  not in glove vocab
788912  not in glove vocab
134a  not in glove vocab
134b  not in glove vocab
lepidochelys  not in glove vocab
cagier  not in glove vocab
89108  not in glove vocab
croceum  not in glove vocab
704301  not in glove vocab
675990  not in glove vocab
4901  not in glove vocab
766567  not in glove vocab
biocontrols  not in glove vocab
13556  not in glove vocab
744554  not in glove vocab
smalstig  not in glove vocab
788471  not in glove vocab


247b  not in glove vocab
reigart  not in glove vocab
apiol  not in glove vocab
butylazo  not in glove vocab
aldicarb  not in glove vocab
ab23  not in glove vocab
lucel  not in glove vocab
diethyltin  not in glove vocab
1location  not in glove vocab
1year  not in glove vocab
8339  not in glove vocab
6560  not in glove vocab
hexachlorophene  not in glove vocab
nofa  not in glove vocab
cresylphosphate  not in glove vocab
za67  not in glove vocab
----------------------------------------------------------------
657 school prayer banned
urumiyeh  not in glove vocab
noncurriculum  not in glove vocab
monafeqin  not in glove vocab
patsey  not in glove vocab
khamene  not in glove vocab
farrakan  not in glove vocab
mergens  not in glove vocab
----------------------------------------------------------------
658 teenage pregnancy
7bn  not in glove vocab
8890  not in glove vocab
642m  not in glove vocab
8bn  not in glove vocab
51c  not in glove vocab
6bn  not in glove vocab
aetcs  not in glove vocab

zvyagilskiy  not in glove vocab
sorzano  not in glove vocab
gerlein  not in glove vocab
volksunion  not in glove vocab
precandidates  not in glove vocab
yemetz  not in glove vocab
ukrinform  not in glove vocab
pliushch  not in glove vocab
----------------------------------------------------------------
671 salvation army benefits
cger  not in glove vocab
sagic  not in glove vocab
continentale  not in glove vocab
6875  not in glove vocab
aslk  not in glove vocab
restof  not in glove vocab
reffin  not in glove vocab
mckorn  not in glove vocab
tilen  not in glove vocab
----------------------------------------------------------------
672 nra membership profile
gunshine  not in glove vocab
owningest  not in glove vocab
wessons  not in glove vocab
graceanne  not in glove vocab
gaudot  not in glove vocab
avtomat  not in glove vocab
supenski  not in glove vocab
morphonios  not in glove vocab
intratec  not in glove vocab
mandall  not in glove vocab
rugers  not in glove vocab
hiers  not in glove

1383c  not in glove vocab
1382c  not in glove vocab
ec3v  not in glove vocab
tecs  not in glove vocab
coloroll  not in glove vocab
1381a  not in glove vocab
1382j  not in glove vocab
1382b  not in glove vocab
3nl  not in glove vocab
----------------------------------------------------------------
685 oscar winner selection
angenieux  not in glove vocab
smotheringly  not in glove vocab
064  not in glove vocab
netalya  not in glove vocab
veraz  not in glove vocab
negoda  not in glove vocab
bennent  not in glove vocab
macminn  not in glove vocab
flanneled  not in glove vocab
chumo  not in glove vocab
aleene  not in glove vocab
----------------------------------------------------------------
686 argentina pegging dollar
niedvaras  not in glove vocab
marketmakers  not in glove vocab
16bn  not in glove vocab
bimonetary  not in glove vocab
31bn  not in glove vocab
flexibilising  not in glove vocab
42bn  not in glove vocab
8bn  not in glove vocab
61bn  not in glove vocab
6bn  not in glove voca

3174  not in glove vocab
0313  not in glove vocab
8306  not in glove vocab
5765  not in glove vocab
15744  not in glove vocab
2964  not in glove vocab
8071  not in glove vocab
26051  not in glove vocab
r4bn  not in glove vocab
3775  not in glove vocab
r3bn  not in glove vocab
r907  not in glove vocab
oromigna  not in glove vocab
6676  not in glove vocab
11852  not in glove vocab
3281  not in glove vocab
13922  not in glove vocab
velopment  not in glove vocab
3185  not in glove vocab
28595  not in glove vocab
5867  not in glove vocab
8258  not in glove vocab
13072  not in glove vocab
evelopment  not in glove vocab
7850  not in glove vocab
10251  not in glove vocab
7395  not in glove vocab
0187  not in glove vocab
3347  not in glove vocab
7702  not in glove vocab
8180  not in glove vocab
31872  not in glove vocab
12141  not in glove vocab
6345  not in glove vocab
3465  not in glove vocab
3773  not in glove vocab
r345  not in glove vocab
elopment  not in glove vocab
6525  not in glove voc

#### Save the result in Json format

In [14]:
SAVE_DIR = '/mnt/scratch2/shnaseri/Experiments/context_query_exp/Robust/exp-terms/'
with open(SAVE_DIR + 'glove_exp_terms_of_topk_docs1.json','w') as output:
    output.write('[')
    for i in range(len(most_sim_terms_per_query) - 1):
        output.write(json.dumps(most_sim_terms_per_query[i]) + ",\n")
    output.write(json.dumps(most_sim_terms_per_query[len(most_sim_terms_per_query) - 1]) + "\n")
    output.write(']')